<center><h1>KIM_GUKIL_HW7</h1></center>

Name: GUK IL KIM
<br>
Github Username: Danielkim12 
<br>
USC ID: 3020867072

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

Import packages

In [72]:
import pandas as pd
import random
import numpy as np
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, silhouette_score
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from IPython.display import display
from imblearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

### (a) Download the Anuran Calls (MFCCs) Data Set

In [73]:
pwd

'/Users/gukilkim_1/Desktop/DSCI_552/hw7/notebook'

In [74]:
df = pd.read_csv('../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')
df

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


### (b) Train a classifier for each label

In [75]:
total_row = len(df) 
random.seed(total_row) # set seed for reproducibility, python random module
split = int(total_row * 0.7) 
train_idx = random.sample(range(total_row), split) # 70% of the data for training
test_idx = [i for i in range(total_row) if i not in train_idx] # 30% of the data
# split the data into training and testing sets
df_train = df.iloc[train_idx, :].reset_index(drop=True)
df_test = df.iloc[test_idx, :].reset_index(drop=True)
# separate features x and labels y for the training set
x_train, x_test = df_train.iloc[:, :-4], df_test.iloc[:, :-4]
# contains multiple labels in one variable 
family_train, genus_train, species_train = df_train['Family'], df_train['Genus'], df_train['Species']
family_test, genus_test, species_test = df_test['Family'], df_test['Genus'], df_test['Species']
print('Training set:', x_train.shape)
print('Testing set:', x_test.shape)

Training set: (5036, 22)
Testing set: (2159, 22)


#### (i) Research

In multi-label classification: 
1. **Exact Match**: It measures the proportin of instances where the predicted labels perfectly match the true labels. Incorrect label in an instance will lead to zero score for that instance. The metric follows 1 - Exact Match Ratio (calculate proportions of instances whose actual value is not equal to predicted value).
2. **Hamnming Score**: It calculates the fraction of incorrect labels across all instances, penalizing both false positives and negatives. This metric is useful for partial correctness evaluation as it provides a ratio of label errors to the the total labels. Accuracy for each instance is defined as the proportion of the predicted correct labels to the total number (predicted and actual) of labels for that instance

#### (ii) Train a SVM for each of the labels

In [76]:
# hamming loss and exact match ratio
def compute_metrics(y_test, y_pred):
    # finds the hamming loss and exact match ratio
    hamming_loss_value = hamming_loss(y_test, y_pred)
    # exact match ratio
    exact_match_ratio = (y_test == y_pred).all(axis=1).mean()
    # round to 4 decimal places
    hamming_loss_value = np.round(hamming_loss_value, 4)
    exact_match_ratio = np.round(exact_match_ratio, 4)
    return hamming_loss_value, exact_match_ratio

In [78]:
def find_best_svm_models(x_train, y_train, x_test, y_test, use_standardization=False, param_grid=None, scoring='accuracy', cv=None):
    # standardize features 
    if use_standardization:
        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        # to identify the type of features used
        feature_type = "Standardized"
    else:
        feature_type = "Raw"
    print(f"\nUsing {feature_type} features:")
    
    # default parameter grid and cross-validation strategy 
    if param_grid is None:
        param_grid = {
            'C': np.logspace(-3, 3, 7),      # Values from 0.001 to 1000
            'gamma': np.logspace(-3, 3, 7),  # Values from 0.001 to 1000
        }
    if cv is None:
        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    models = {}
    y_pred = pd.DataFrame(index=y_test.index)
    # train models for each label
    for label in y_train.columns:
        svc = SVC(kernel='rbf', decision_function_shape='ovr')
        grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=cv, n_jobs=1, scoring=scoring)
        grid_search.fit(x_train, y_train[label])
        # Get the best estimator and store it in the models dictionary
        best_model = grid_search.best_estimator_
        models[label] = best_model
        print(f"Best parameters for {label}: {grid_search.best_params_}")
        # predict on the test set and store predictions
        y_pred[label] = best_model.predict(x_test)
        print(f"Classification report for {label}:")
        print(classification_report(y_test[label], y_pred[label]))
        acc = accuracy_score(y_test[label], y_pred[label])
        print(f"Accuracy for {label}: {acc:.4f}")
    # Compute multi-label metrics using compute_metrics
    hamming_loss_value, exact_match_ratio = compute_metrics(y_test, y_pred)
    print(f"Hamming Loss: {hamming_loss_value}")
    print(f"Exact Match Ratio: {exact_match_ratio}")
    return models, y_pred


In [79]:
# 10-fold cross-validation
param_grid = {
    'C': np.logspace(-2, 2, 5),        # [0.01, 0.1, 1, 10, 100]
    'gamma': np.logspace(-4, -1, 4),   # [0.0001, 0.001, 0.01, 0.1]
}
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=69)

# run function for each label
def run_model_for_label(label_name, y_train, y_test):
    print(f"\nLabel: {label_name}")
    # convert the label to binary
    y_train_df = y_train.to_frame()
    y_test_df = y_test.to_frame()
    # call the function
    models, predictions = find_best_svm_models(
        x_train, y_train_df, x_test, y_test_df,
        use_standardization=False, param_grid=param_grid, scoring='f1_weighted', cv=cv
    )
    return models, predictions

In [80]:
family_model, family_predictions = run_model_for_label("Family", family_train, family_test)


Label: Family

Using Raw features:
Best parameters for Family: {'C': 100.0, 'gamma': 0.1}
Classification report for Family:
                 precision    recall  f1-score   support

      Bufonidae       0.74      0.93      0.82        15
  Dendrobatidae       0.99      0.98      0.98       154
        Hylidae       0.99      0.97      0.98       673
Leptodactylidae       0.99      0.99      0.99      1317

       accuracy                           0.98      2159
      macro avg       0.92      0.97      0.94      2159
   weighted avg       0.99      0.98      0.98      2159

Accuracy for Family: 0.9847
Hamming Loss: 0.0153
Exact Match Ratio: 0.9847


In [81]:
genus_model, genus_predictions = run_model_for_label("Genus", genus_train, genus_test)


Label: Genus

Using Raw features:


Best parameters for Genus: {'C': 100.0, 'gamma': 0.1}
Classification report for Genus:
               precision    recall  f1-score   support

    Adenomera       0.99      1.00      0.99      1240
     Ameerega       0.99      0.97      0.98       154
Dendropsophus       0.94      0.92      0.93        91
    Hypsiboas       0.98      0.99      0.98       479
Leptodactylus       0.96      0.97      0.97        77
Osteocephalus       1.00      0.85      0.92        47
     Rhinella       0.93      0.93      0.93        15
       Scinax       1.00      0.95      0.97        56

     accuracy                           0.98      2159
    macro avg       0.97      0.95      0.96      2159
 weighted avg       0.98      0.98      0.98      2159

Accuracy for Genus: 0.9838
Hamming Loss: 0.0162
Exact Match Ratio: 0.9838


In [82]:
species_model, species_predictions = run_model_for_label("Species", species_train, species_test)


Label: Species

Using Raw features:
Best parameters for Species: {'C': 100.0, 'gamma': 0.1}
Classification report for Species:
                        precision    recall  f1-score   support

        AdenomeraAndre       0.99      0.97      0.98       182
AdenomeraHylaedactylus       0.99      1.00      1.00      1058
    Ameeregatrivittata       0.99      0.97      0.98       154
            HylaMinuta       0.94      0.92      0.93        91
  HypsiboasCinerascens       0.97      0.99      0.98       153
     HypsiboasCordobae       0.99      0.98      0.98       326
   LeptodactylusFuscus       0.95      0.99      0.97        77
 OsteocephalusOophagus       0.91      0.89      0.90        47
     Rhinellagranulosa       0.93      0.93      0.93        15
           ScinaxRuber       1.00      0.96      0.98        56

              accuracy                           0.98      2159
             macro avg       0.97      0.96      0.96      2159
          weighted avg       0.98     

In [84]:
def evaluate_combined_metrics(y_true_dict, y_pred_dict):
    # concatenate the true and predicted labels for each category into binary format
    y_true_combined = pd.concat([pd.get_dummies(y_true_dict['Family']), 
                                 pd.get_dummies(y_true_dict['Genus']), 
                                 pd.get_dummies(y_true_dict['Species'])], axis=1)
    y_pred_combined = pd.concat([pd.get_dummies(y_pred_dict['Family']), 
                                 pd.get_dummies(y_pred_dict['Genus']), 
                                 pd.get_dummies(y_pred_dict['Species'])], axis=1)
    # compute Hamming Loss and Exact Match Ratio
    hamming_loss_value = hamming_loss(y_true_combined, y_pred_combined)
    exact_match_ratio = (y_true_combined == y_pred_combined).all(axis=1).mean()
    print("\nCombined Evaluation Metrics:")
    print(f"Hamming Loss: {hamming_loss_value:.4f}")
    print(f"Exact Match Ratio: {exact_match_ratio:.4f}")
    return 

# dictionary to hold true and predicted values
y_true_dict = {'Family': family_test, 'Genus': genus_test, 'Species': species_test}
y_pred_dict = {'Family': family_predictions['Family'], 'Genus': genus_predictions['Genus'], 'Species': species_predictions['Species']}

print("Evaluation Metrics for Individual Labels:")
evaluate_combined_metrics(y_true_dict, y_pred_dict)

Evaluation Metrics for Individual Labels:

Combined Evaluation Metrics:
Hamming Loss: 0.0043
Exact Match Ratio: 0.9731


### for standardized data values

In [85]:
def run_model_for_label(label_name, y_train, y_test):
    print(f"\nLabel: {label_name}")    
    y_train_df = y_train.to_frame()
    y_test_df = y_test.to_frame()
    # standardization
    models, predictions = find_best_svm_models(
        x_train, y_train_df, x_test, y_test_df,
        use_standardization=True, param_grid=param_grid, scoring='f1_weighted', cv=cv
    )
    return models, predictions

In [86]:
family_model_std, family_predictions_std = run_model_for_label("Family", family_train, family_test)


Label: Family

Using Standardized features:
Best parameters for Family: {'C': 100.0, 'gamma': 0.01}
Classification report for Family:
                 precision    recall  f1-score   support

      Bufonidae       0.88      0.93      0.90        15
  Dendrobatidae       0.99      0.99      0.99       154
        Hylidae       0.99      0.98      0.99       673
Leptodactylidae       0.99      0.99      0.99      1317

       accuracy                           0.99      2159
      macro avg       0.96      0.98      0.97      2159
   weighted avg       0.99      0.99      0.99      2159

Accuracy for Family: 0.9898
Hamming Loss: 0.0102
Exact Match Ratio: 0.9898


In [87]:
genus_model_std, genus_predictions_std = run_model_for_label("Genus", genus_train, genus_test)


Label: Genus

Using Standardized features:
Best parameters for Genus: {'C': 10.0, 'gamma': 0.1}
Classification report for Genus:
               precision    recall  f1-score   support

    Adenomera       0.99      1.00      1.00      1240
     Ameerega       1.00      1.00      1.00       154
Dendropsophus       1.00      0.95      0.97        91
    Hypsiboas       0.96      1.00      0.98       479
Leptodactylus       1.00      0.97      0.99        77
Osteocephalus       1.00      0.81      0.89        47
     Rhinella       1.00      0.87      0.93        15
       Scinax       1.00      0.95      0.97        56

     accuracy                           0.99      2159
    macro avg       0.99      0.94      0.97      2159
 weighted avg       0.99      0.99      0.99      2159

Accuracy for Genus: 0.9880
Hamming Loss: 0.012
Exact Match Ratio: 0.988


In [88]:
species_model_std, species_predictions_std = run_model_for_label("Species", species_train, species_test)


Label: Species

Using Standardized features:
Best parameters for Species: {'C': 100.0, 'gamma': 0.01}
Classification report for Species:
                        precision    recall  f1-score   support

        AdenomeraAndre       0.98      0.98      0.98       182
AdenomeraHylaedactylus       0.99      1.00      1.00      1058
    Ameeregatrivittata       0.99      0.99      0.99       154
            HylaMinuta       0.96      0.95      0.95        91
  HypsiboasCinerascens       0.97      0.99      0.98       153
     HypsiboasCordobae       0.99      0.98      0.99       326
   LeptodactylusFuscus       0.97      0.99      0.98        77
 OsteocephalusOophagus       0.95      0.89      0.92        47
     Rhinellagranulosa       0.94      1.00      0.97        15
           ScinaxRuber       1.00      0.98      0.99        56

              accuracy                           0.99      2159
             macro avg       0.98      0.98      0.98      2159
          weighted avg      

In [90]:
y_true_dict = {'Family': family_test, 'Genus': genus_test, 'Species': species_test}
y_pred_dict = {'Family': family_predictions_std['Family'], 'Genus': genus_predictions_std['Genus'], 'Species': species_predictions_std['Species']}

print("Evaluation Metrics for Standardized Features:")
evaluate_combined_metrics(y_true_dict, y_pred_dict)

Evaluation Metrics for Standardized Features:

Combined Evaluation Metrics:
Hamming Loss: 0.0031
Exact Match Ratio: 0.9801


### observation: 
As evident from the evaluation, standardizing the data improves accuracy and reduces Hamming Loss (Exact Match Ratio: 0.9801, Hamming Loss: 0.0031) compared to using raw data (Exact Match Ratio: 0.9731, Hamming Loss: 0.0043).

#### (iii) Repeat 1(b)ii with L1-penalized SVMs

### values are standardized

In [125]:
method = "Support Vector Classifier with L1-penalty"
print("Method: ", method)
param_grid = {'C': np.logspace(-3, 2, 6)} # [0.001, 0.01, 0.1, 1, 10, 100] 
cv = StratifiedKFold(n_splits=10, random_state=69, shuffle=True)

# staandardization for linear SVC
use_standardization = True
if use_standardization:
    scaler = StandardScaler()
    x_train_std = scaler.fit_transform(x_train)
    x_test_std = scaler.transform(x_test)
else:
    x_train_std, x_test_std = x_train, x_test

def run_linear_svc_for_label(label_name, y_train, y_test):
    print(f"\nLabel: {label_name}")
    y_train_df = y_train.to_frame()
    y_test_df = y_test.to_frame()
    classifier = LinearSVC(penalty='l1', dual=False, max_iter=10000)
    # standardize attribute
    model = find_best_svm_models(
        x_train_std, y_train_df, x_test_std, y_test_df,
        use_standardization=True, param_grid=param_grid, scoring='f1_weighted', cv=cv
    )
    return model

Method:  Support Vector Classifier with L1-penalty


In [126]:
family_model_L1, family_predictions_L1 = run_linear_svc_for_label("Family", family_train, family_test)


Label: Family

Using Standardized features:
Best parameters for Family: {'C': 100.0}
Classification report for Family:
                 precision    recall  f1-score   support

      Bufonidae       0.93      0.93      0.93        15
  Dendrobatidae       0.99      0.99      0.99       154
        Hylidae       0.99      0.99      0.99       673
Leptodactylidae       1.00      0.99      1.00      1317

       accuracy                           0.99      2159
      macro avg       0.98      0.98      0.98      2159
   weighted avg       0.99      0.99      0.99      2159

Accuracy for Family: 0.9926
Hamming Loss: 0.0074
Exact Match Ratio: 0.9926


In [127]:
genus_model_L1, genus_predictions_L1 = run_linear_svc_for_label("Genus", genus_train, genus_test)


Label: Genus

Using Standardized features:


Best parameters for Genus: {'C': 10.0}
Classification report for Genus:
               precision    recall  f1-score   support

    Adenomera       0.99      1.00      0.99      1240
     Ameerega       1.00      1.00      1.00       154
Dendropsophus       0.98      0.98      0.98        91
    Hypsiboas       0.98      0.99      0.98       479
Leptodactylus       0.99      0.97      0.98        77
Osteocephalus       1.00      0.85      0.92        47
     Rhinella       1.00      0.93      0.97        15
       Scinax       1.00      0.96      0.98        56

     accuracy                           0.99      2159
    macro avg       0.99      0.96      0.98      2159
 weighted avg       0.99      0.99      0.99      2159

Accuracy for Genus: 0.9889
Hamming Loss: 0.0111
Exact Match Ratio: 0.9889


In [128]:
species_model_L1, species_predictions_L1 = run_linear_svc_for_label("Species", species_train, species_test)


Label: Species

Using Standardized features:
Best parameters for Species: {'C': 100.0}
Classification report for Species:
                        precision    recall  f1-score   support

        AdenomeraAndre       0.99      0.98      0.99       182
AdenomeraHylaedactylus       0.99      1.00      1.00      1058
    Ameeregatrivittata       1.00      1.00      1.00       154
            HylaMinuta       0.97      0.96      0.96        91
  HypsiboasCinerascens       0.97      0.99      0.98       153
     HypsiboasCordobae       0.98      0.99      0.99       326
   LeptodactylusFuscus       0.99      1.00      0.99        77
 OsteocephalusOophagus       1.00      0.87      0.93        47
     Rhinellagranulosa       1.00      0.93      0.97        15
           ScinaxRuber       1.00      0.98      0.99        56

              accuracy                           0.99      2159
             macro avg       0.99      0.97      0.98      2159
          weighted avg       0.99      0.99

In [129]:
y_true_dict = {'Family': family_test, 'Genus': genus_test, 'Species': species_test}
y_pred_dict = {'Family': family_predictions_L1['Family'], 'Genus': genus_predictions_L1['Genus'], 'Species': species_predictions_L1['Species']}

print("Evaluation metrics for Linear SVC with L1-penalty:")
evaluate_combined_metrics(y_true_dict, y_pred_dict)

Evaluation metrics for Linear SVC with L1-penalty:

Combined Evaluation Metrics:
Hamming Loss: 0.0026
Exact Match Ratio: 0.9852


(0.0025685292012295256, 0.9851783232978231)

#### (iv) Repeat 1(b)iii by using SMOTE or any other method for imbalance

In [144]:
# Define the parameter grid and cross-validation strategy
param_grid = {'classification__C': np.logspace(-4, 4, 9)} # [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000] wider range for flexibility
cv = StratifiedKFold(n_splits=5, random_state=69, shuffle=True)  # Reduced folds to 5 to save memory

In [142]:
def run_linear_svc_with_smote_for_label(label_name, y_train, y_test):
    print(f"\nLabel: {label_name}")
    # Ensure `y_train` and `y_test` are DataFrames
    y_train_df = y_train.to_frame() if isinstance(y_train, pd.Series) else y_train
    y_test_df = y_test.to_frame() if isinstance(y_test, pd.Series) else y_test
    
    # Define a pipeline with SMOTE and LinearSVC
    pipeline = Pipeline([
        ('sampling', SMOTE(random_state=42)),
        ('classification', LinearSVC(penalty='l1', dual=False, max_iter=10000))  # Lower max_iter to save resources
    ])
    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=cv, scoring='f1_weighted', n_jobs=1)
    grid_search.fit(x_train_std, y_train_df)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(x_test_std)
    y_pred_df = pd.DataFrame(y_pred, index=y_test_df.index, columns=[label_name])
    # Print the best parameters and evaluation metrics
    print(f"Best parameters for {label_name}: {grid_search.best_params_}")
    print(f"Classification report for {label_name}:")
    print(classification_report(y_test_df, y_pred_df))
    acc = accuracy_score(y_test_df, y_pred_df)
    print(f"Accuracy for {label_name}: {acc:.4f}")
    hamming_loss_value, exact_match_ratio = compute_metrics(y_test_df, y_pred_df)
    print(f"Hamming Loss: {hamming_loss_value}")
    print(f"Exact Match Ratio: {exact_match_ratio}")
    return best_model, y_pred_df

In [143]:
family_model_smote, family_predictions_smote = run_linear_svc_with_smote_for_label("Family", family_train, family_test)


Label: Family
Best parameters for Family: {'classification__C': 100.0}
Classification report for Family:
                 precision    recall  f1-score   support

      Bufonidae       0.24      0.93      0.38        15
  Dendrobatidae       0.80      0.98      0.88       154
        Hylidae       0.95      0.87      0.91       673
Leptodactylidae       0.96      0.95      0.96      1317

       accuracy                           0.93      2159
      macro avg       0.74      0.93      0.78      2159
   weighted avg       0.94      0.93      0.93      2159

Accuracy for Family: 0.9259
Hamming Loss: 0.0741
Exact Match Ratio: 0.9259


In [145]:
genus_model_smote, genus_predictions_smote = run_linear_svc_with_smote_for_label("Genus", genus_train, genus_test)


Label: Genus


Best parameters for Genus: {'classification__C': 10.0}
Classification report for Genus:
               precision    recall  f1-score   support

    Adenomera       0.98      0.93      0.96      1240
     Ameerega       0.83      0.93      0.87       154
Dendropsophus       0.69      0.87      0.77        91
    Hypsiboas       0.97      0.91      0.94       479
Leptodactylus       0.92      0.92      0.92        77
Osteocephalus       0.56      0.74      0.64        47
     Rhinella       0.28      0.87      0.42        15
       Scinax       0.87      0.98      0.92        56

     accuracy                           0.92      2159
    macro avg       0.76      0.89      0.80      2159
 weighted avg       0.94      0.92      0.92      2159

Accuracy for Genus: 0.9185
Hamming Loss: 0.0815
Exact Match Ratio: 0.9185


In [146]:
species_model_smote, species_predictions_smote = run_linear_svc_with_smote_for_label("Species", species_train, species_test)


Label: Species


Best parameters for Species: {'classification__C': 1.0}
Classification report for Species:
                        precision    recall  f1-score   support

        AdenomeraAndre       0.97      0.91      0.94       182
AdenomeraHylaedactylus       0.99      1.00      0.99      1058
    Ameeregatrivittata       0.93      0.92      0.92       154
            HylaMinuta       0.84      0.89      0.87        91
  HypsiboasCinerascens       0.92      0.93      0.93       153
     HypsiboasCordobae       0.95      0.91      0.93       326
   LeptodactylusFuscus       0.93      0.91      0.92        77
 OsteocephalusOophagus       0.80      0.70      0.75        47
     Rhinellagranulosa       0.41      0.93      0.57        15
           ScinaxRuber       0.89      0.96      0.92        56

              accuracy                           0.95      2159
             macro avg       0.86      0.91      0.87      2159
          weighted avg       0.96      0.95      0.95      2159

Accuracy f

In [148]:
y_true_dict = {'Family': family_test, 'Genus': genus_test, 'Species': species_test}
y_pred_dict = {'Family': family_predictions_smote['Family'], 'Genus': genus_predictions_smote['Genus'], 'Species': species_predictions_smote['Species']}

print("Evaluation metrics for Linear SVC with L1-penalty and SMOTE for Imbalance:")
evaluate_combined_metrics(y_true_dict, y_pred_dict)

Evaluation metrics for Linear SVC with L1-penalty and SMOTE for Imbalance:

Combined Evaluation Metrics:
Hamming Loss: 0.0187
Exact Match Ratio: 0.8652


(0.018653416985978357, 0.8652153774895786)

### Observation: 
As shown above, when comparing the Support Vector Classifier with L1-penalty, Linear SVC with L1-penalty, and Linear SVC with L1-penalty combined with SMOTE, the model with L1-penalty and SMOTE achieves a lower exact match ratio (accuracy score) of 0.8652 but also a lower Hamming loss of 0.0187. In contrast, the model with only L1-penalty achieves a higher exact match ratio of 0.9852 but a higher Hamming loss of 0.0026.

In [149]:
evaluation_data = {
    "Method": [
        "Linear SVC with L1-penalty and SMOTE",
        "Linear SVC with L1-penalty",
        "Standardized Features",
        "Individual Labels (Raw Features)"
    ],
    "Hamming Loss": [
        0.0187,
        0.0026,
        0.0031,
        0.0043
    ],
    "Exact Match Ratio": [
        0.8652,
        0.9852,
        0.9801,
        0.9731
    ]
}
evaluation_report = pd.DataFrame(evaluation_data)
# Display the DataFrame
print("Evaluation Report:")
display(evaluation_report)

Evaluation Report:


,Method,Hamming Loss,Exact Match Ratio
0,Linear SVC with L1-penalty and SMOTE,0.0187,0.8652
1,Linear SVC with L1-penalty,0.0026,0.9852
2,Standardized Features,0.0031,0.9801
3,Individual Labels (Raw Features),0.0043,0.9731


### Report Combined
From the chart above, wwe can fnd that Linear SVC with L1-penalty without SMOTE is optimal for accuracy, while SMOTE is useful for handling class imbalance at a minor accuracy trade-off. Standardizing features generally enhances model performance.

## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set

### (a) Use k-means clustering, (b) Determine which family is the majority, (c) Calculate the average Hamming distance, Hamming score, and Hamming loss [combined]

In [22]:
# find the best k for KMeans clustering, fixed 
def find_best_k (data, k_range=(2, 50), random_state=None):
    best_k = k_range[0]
    # silhouette Score can be negative 
    best_score = -1  
    # loop through the range of k values
    for k in range(k_range[0], k_range[1] + 1):
        kmeans = KMeans(n_clusters=k, random_state=random_state)
        labels = kmeans.fit_predict(data)
        score = silhouette_score(data, labels)
        if score > best_score:
            best_k = k
            best_score = score
    return best_k, best_score

In [23]:
def majority_vote_labels(cluster_labels, true_labels):
    clusters = np.unique(cluster_labels)
    cluster_majorities = []
    for cluster in clusters:
        indices = np.where(cluster_labels == cluster)[0]
        cluster_true_labels = true_labels.iloc[indices]
        # find the majority label for each column in the cluster
        majority_labels = cluster_true_labels.mode().iloc[0]
        # create a dictionary with the cluster number and majority labels
        majority_dict = {'Cluster': cluster}
        for col in true_labels.columns:
            majority_dict[col] = majority_labels[col]
        cluster_majorities.append(majority_dict)
    # convert list of dictionaries to DataFrame
    cluster_majorities_df = pd.DataFrame(cluster_majorities)
    return cluster_majorities_df

In [24]:
# hamming loss and exact match ratio and hamming distance
def compute_all_metrics(cluster_majorities, cluster_labels, true_labels):
    total_labels = true_labels.shape[0] * true_labels.shape[1]
    mismatches = 0
    for idx, row in cluster_majorities.iterrows():
        cluster = row['Cluster']
        majority_labels = row[true_labels.columns]
        indices = np.where(cluster_labels == cluster)[0]
        cluster_true_labels = true_labels.iloc[indices]
        # Count mismatches for this cluster
        mismatches += (cluster_true_labels != majority_labels.values).sum().sum()
    # Hamming Loss: Fraction of incorrect labels
    hamming_loss_value = mismatches / total_labels
    # Hamming Distance: Average number of mismatches per sample
    hamming_distance = mismatches / true_labels.shape[0]
    # Round the results
    hamming_loss_value = np.round(hamming_loss_value, 4)
    hamming_distance = np.round(hamming_distance, 4)
    return hamming_distance, hamming_loss_value

In [28]:
def monte_carlo(y_true, X_data, num_simulations=50, k_range=(2, 50)):
    hamming_distances = [] # store all hamming distances
    hamming_losses = [] # store all hamming losses
    all_cluster_majorities = [] # store all cluster majorities
    # 50 iterations of the simulation
    for iteration in range(num_simulations):
        print(f"Simulation {iteration + 1}/{num_simulations}")
        random_state = iteration  # For reproducibility
        best_k, best_score = find_best_k(X_data, k_range=k_range, random_state=random_state) # best k and silhouette score
        kmeans = KMeans(n_clusters=best_k, random_state=random_state) # clustering with best k
        cluster_labels = kmeans.fit_predict(X_data)
        cluster_majorities = majority_vote_labels(cluster_labels, y_true)
        # append cluster majorities to the list for later analysis
        if not cluster_majorities.empty:
            all_cluster_majorities.append(cluster_majorities)
        # display tables for each iteration
        display(cluster_majorities) # display the cluster majorities
        print("\n")
        # compute hamming distance and hamming loss
        hamming_distance, hamming_loss_value = compute_all_metrics(cluster_majorities, cluster_labels, y_true)
        hamming_distances.append(hamming_distance)
        hamming_losses.append(hamming_loss_value)
        print(f"Best k: {best_k} with Silhouette Score: {best_score:.4f}")
        print(f"Hamming Distance: {hamming_distance}")
        print(f"Hamming Loss: {hamming_loss_value}\n")

    # combine all cluster majorities into a single DataFrame
    if all_cluster_majorities:
        cluster_majorities_df = pd.concat(all_cluster_majorities, ignore_index=True)
        print("Combined Cluster Majorities across all iterations:")
    else:
        print("No cluster majorities were generated across iterations.")
        cluster_majorities_df = pd.DataFrame()

    # calculate avg hamming distance, std, hamming score, and avg hamming loss
    avg_hamming_distance = np.mean(hamming_distances)
    std_hamming_distance = np.std(hamming_distances)
    hamming_score = 1 - avg_hamming_distance
    avg_hamming_loss = np.mean(hamming_losses)
    # final analysis
    print(f"Average Hamming Distance over {num_simulations} simulations: {avg_hamming_distance:.4f}")
    print(f"Standard Deviation of Hamming Distances: {std_hamming_distance:.4f}")
    print(f"Hamming Score: {hamming_score:.4f}")
    print(f"Average Hamming Loss: {avg_hamming_loss:.4f}")
    return avg_hamming_distance, std_hamming_distance, cluster_majorities_df

In [29]:
# exclude last 4 columns
X_data = df.iloc[:, :-4]
y_true = df[['Family', 'Genus', 'Species']]
# Run the Monte Carlo simulation
average_distance, std_distance, cluster_majorities_df = monte_carlo(y_true, X_data, num_simulations=50, k_range=(2, 50))


Simulation 1/50


,Cluster,Family,Genus,Species
0,0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,1,Dendrobatidae,Ameerega,Ameeregatrivittata
2,2,Hylidae,Hypsiboas,HypsiboasCinerascens
3,3,Hylidae,Hypsiboas,HypsiboasCordobae




Best k: 4 with Silhouette Score: 0.3789
Hamming Distance: 0.6653
Hamming Loss: 0.2218

Simulation 2/50


,Cluster,Family,Genus,Species
0,0,Hylidae,Hypsiboas,HypsiboasCinerascens
1,1,Dendrobatidae,Ameerega,Ameeregatrivittata
2,2,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
3,3,Hylidae,Hypsiboas,HypsiboasCordobae




Best k: 4 with Silhouette Score: 0.3788
Hamming Distance: 0.6673
Hamming Loss: 0.2224

Simulation 3/50


...



Best k: 4 with Silhouette Score: 0.3788
Hamming Distance: 0.6673
Hamming Loss: 0.2224

Simulation 49/50


,Cluster,Family,Genus,Species
0,0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,1,Hylidae,Hypsiboas,HypsiboasCordobae
2,2,Hylidae,Hypsiboas,HypsiboasCinerascens
3,3,Hylidae,Hypsiboas,HypsiboasCordobae




Best k: 4 with Silhouette Score: 0.3747
Hamming Distance: 0.8402
Hamming Loss: 0.2801

Simulation 50/50


,Cluster,Family,Genus,Species
0,0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,1,Hylidae,Hypsiboas,HypsiboasCinerascens
2,2,Dendrobatidae,Ameerega,Ameeregatrivittata
3,3,Hylidae,Hypsiboas,HypsiboasCordobae




Best k: 4 with Silhouette Score: 0.3788
Hamming Distance: 0.6673
Hamming Loss: 0.2224

Combined Cluster Majorities across all iterations:
Average Hamming Distance over 50 simulations: 0.6712
Standard Deviation of Hamming Distances: 0.0310
Hamming Score: 0.3288
Average Hamming Loss: 0.2237


+ as shown under the monte_carlo() average Hamming score follows (1 - avg_hamming_loss) = 0.7763

## 3. ISLR 12.6.2

### Will be attached as seperate pdf 

### Citation
- https://scikit-learn.org/dev/modules/generated/sklearn.svm.SVC.html
- https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.hamming_loss.html 
- https://scikit-learn.org/dev/modules/generated/sklearn.metrics.accuracy_score.html
- https://scikit-learn.org/1.5/modules/grid_search.html
- https://scikit-learn.org/dev/modules/generated/sklearn.svm.LinearSVC.html
- https://stackoverflow.com/questions/72749144/scikit-learn-pipeline-with-smote
- https://scikit-learn.org/dev/modules/generated/sklearn.metrics.silhouette_score.html

### ChatGPT Prompt and Responses

Prompt: I am trying to train SVM for each label using gaussian 10 fold cross validation and one versus all classifiers. I want two portion of data one as raw and one as standardized using scaler (I would like to have if-condition to either convert data into standardized data or not). give me syntax for this. 

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import pandas as pd

# Assuming X is your features DataFrame and y is a DataFrame with each label as a column
# X, y = <load your data>

# Define the parameter grid for Gaussian (RBF) kernel SVM
param_grid = {
    'C': np.logspace(-3, 3, 7),      # Regularization parameter
    'gamma': np.logspace(-3, 3, 7)   # Kernel coefficient
}

# Set up 10-fold cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Dictionary to store results for each label
results = {}

# Loop over each label in the y DataFrame
for label_name in y.columns:
    print(f"\nTraining SVM for label: {label_name}")
    
    # Extract label-specific target values
    y_label = y[label_name]
    
    # If-condition to standardize data
    use_standardization = True  # Set this to False to use raw data
    if use_standardization:
        scaler = StandardScaler()
        X_transformed = scaler.fit_transform(X)
        feature_type = "Standardized"
    else:
        X_transformed = X
        feature_type = "Raw"
    
    print(f"\nUsing {feature_type} Features for {label_name}")
    
    # Define the model
    svc = SVC(kernel='rbf', decision_function_shape='ovr')
    
    # Set up GridSearchCV with 10-fold cross-validation
    grid_search = GridSearchCV(estimator=svc, param_grid={'C': param_grid['C'], 'gamma': param_grid['gamma']}, 
                               cv=cv, scoring='accuracy', n_jobs=-1)
    
    # Fit the model
    grid_search.fit(X_transformed, y_label)
    
    # Get the best parameters and best cross-validation score
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    print("Best Parameters:", best_params)
    print("Best Cross-Validation Score:", best_score)
    
    # Predict on the entire dataset with the best model
    y_pred = grid_search.best_estimator_.predict(X_transformed)
    
    # Display classification report
    print(f"\nClassification Report for {label_name} ({feature_type} Features):")
    print(classification_report(y_label, y_pred))
    
    # Store results
    results[label_name] = {
        'feature_type': feature_type,
        'best_params': best_params,
        'best_score': best_score,
        'classification_report': classification_report(y_label, y_pred, output_dict=True)
    }

# Optionally, convert results to a DataFrame for better readability
results_df = pd.DataFrame(results)
print("\nSummary of Results:")
print(results_df)


Prompt: Given this CSV ('../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv'), what would be good hyperparameter

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': np.logspace(-2, 2, 5),        # [0.01, 0.1, 1, 10, 100]
    'gamma': np.logspace(-4, -1, 4),   # [0.0001, 0.001, 0.01, 0.1]
    'kernel': ['rbf']
}

# Example GridSearchCV setup
model = SVC()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)  # Assuming X and y are your feature matrix and target variable

Prompt: when data is standardized, i want to find L1-penalty and L1-penalty with SMOTE. what is a good param_grid (hyperparam) for this? 

In [ ]:
# without SMOTE
param_grid = {'C': np.logspace(-3, 2, 6)}  # [0.001, 0.01, 0.1, 1, 10, 100]

# with SMOTE 
param_grid_smote = {'classification__C': np.logspace(-4, 4, 9)}  # [0.0001, ..., 10000]

Prompt: tell me how i can evaluate scores for hamming loss and exact match score, the run command syntax follows: species_model, species_predictions = run_model_for_label("Species", species_train, species_test), genus, and family as well 

In [ ]:
from sklearn.metrics import hamming_loss

def evaluate_scores(y_true, y_pred):
    # Compute Hamming Loss
    hamming_loss_value = hamming_loss(y_true, y_pred)
    # Compute Exact Match Ratio
    exact_match_ratio = (y_true == y_pred).all(axis=1).mean()
    # Round the scores for readability
    hamming_loss_value = round(hamming_loss_value, 4)
    exact_match_ratio = round(exact_match_ratio, 4)
    print(f"Hamming Loss: {hamming_loss_value}")
    print(f"Exact Match Ratio: {exact_match_ratio}")
    return hamming_loss_value, exact_match_ratio

# Running models for each label and getting predictions
family_model, family_predictions = run_model_for_label("Family", family_train, family_test)
genus_model, genus_predictions = run_model_for_label("Genus", genus_train, genus_test)
species_model, species_predictions = run_model_for_label("Species", species_train, species_test)

# Evaluating the scores for each label
print("\nEvaluation for Family:")
family_hamming_loss, family_exact_match_ratio = evaluate_scores(family_test, family_predictions)

print("\nEvaluation for Genus:")
genus_hamming_loss, genus_exact_match_ratio = evaluate_scores(genus_test, genus_predictions)

print("\nEvaluation for Species:")
species_hamming_loss, species_exact_match_ratio = evaluate_scores(species_test, species_predictions)

Prompt: I want to implement monte carlo function (clustering with best k) that uses hamming loss and hamming distance, thus I want to display table using display(). give me syntax for it 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import hamming_loss, silhouette_score
from sklearn.preprocessing import LabelBinarizer
from IPython.display import display

def monte_carlo_clustering(y_true, X_data, num_simulations=50, k_range=(2, 50)):
    """
    Perform Monte Carlo simulation to find the best k for clustering and evaluate 
    the clustering using Hamming Loss and Hamming Distance.

    Parameters:
    - y_true: DataFrame or array-like, true labels for evaluation
    - X_data: DataFrame or array-like, features for clustering
    - num_simulations: int, number of Monte Carlo iterations
    - k_range: tuple, range of k values to evaluate for clustering
    
    Returns:
    - results_df: DataFrame, summarized results of the Monte Carlo simulations
    """
    # Storage for results
    results = []

    for iteration in range(num_simulations):
        print(f"Simulation {iteration + 1}/{num_simulations}")
        random_state = iteration  # For reproducibility
        
        # Step 1: Find the best k using silhouette score
        best_k, best_score = find_best_k(X_data, k_range=k_range, random_state=random_state)
        
        # Step 2: Perform clustering with the best k
        kmeans = KMeans(n_clusters=best_k, random_state=random_state)
        cluster_labels = kmeans.fit_predict(X_data)
        
        # Step 3: Evaluate clustering using majority vote
        cluster_majorities = majority_vote_labels(cluster_labels, y_true)
        
        # Step 4: Compute Hamming Loss and Hamming Distance
        hamming_distance, hamming_loss_value = compute_clustering_metrics(cluster_majorities, cluster_labels, y_true)
        
        # Store results for this iteration
        results.append({
            'Iteration': iteration + 1,
            'Best k': best_k,
            'Silhouette Score': round(best_score, 4),
            'Hamming Distance': round(hamming_distance, 4),
            'Hamming Loss': round(hamming_loss_value, 4)
        })
    
    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    
    # Display results
    print("\nMonte Carlo Clustering Results:")
    display(results_df)
    
    return results_df

# Supporting functions
def find_best_k(X_data, k_range, random_state):
    """
    Find the best k for clustering using the silhouette score.
    """
    best_k, best_score = 2, -1
    for k in range(k_range[0], k_range[1] + 1):
        kmeans = KMeans(n_clusters=k, random_state=random_state)
        cluster_labels = kmeans.fit_predict(X_data)
        score = silhouette_score(X_data, cluster_labels)
        if score > best_score:
            best_k, best_score = k, score
    return best_k, best_score

def majority_vote_labels(cluster_labels, y_true):
    """
    Determine the majority label for each cluster.
    """
    cluster_majorities = {}
    for cluster in np.unique(cluster_labels):
        indices = np.where(cluster_labels == cluster)[0]
        cluster_samples = y_true.iloc[indices]
        majority_label = cluster_samples.mode().iloc[0]  # Get majority label for each column
        cluster_majorities[cluster] = majority_label
    return cluster_majorities

def compute_clustering_metrics(cluster_majorities, cluster_labels, y_true):
    """
    Compute Hamming Loss and Hamming Distance for clustering predictions.
    """
    # Convert cluster predictions to majority labels
    y_pred = pd.DataFrame(index=y_true.index, columns=y_true.columns)
    for idx, cluster in enumerate(cluster_labels):
        y_pred.iloc[idx, :] = cluster_majorities[cluster]
    
    # Compute metrics
    hamming_distance = (y_true.values != y_pred.values).sum() / y_true.size
    hamming_loss_value = hamming_loss(y_true, y_pred)
    return hamming_distance, hamming_loss_value

# Example usage
# Assuming y_true (labels) and X_data (features) are loaded
# results_df = monte_carlo_clustering(y_true, X_data, num_simulations=50, k_range=(2, 50))